In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyreadstat as stat
import seaborn as sns
from pathlib import Path
import re

In [2]:
# 0) Setup
data_dir = Path("/Users/adamcartwright/ncerdc/Student Data/Testing Data")
years    = [2017, 2018, 2019, 2020, 2021, 2022, 2023]

# 1) Map raw names → cleaned names (including reporting_year → year)
rename_map = {
    'sat_erw_score_hc': 'sat_ebrw_score_hc',
    'sat_erw_score_mr': 'sat_ebrw_score_mr',
    'sat_ctsh_ss_hc':   'sat_math_score_hc',
    'sat_ctsh_ss_mr':   'sat_math_score_mr',
    'ital':             'italgr',
    'reporting_year':   'year'
}

# 2) Build master column list
seen = []
for yr in years:
    hdr = pd.read_csv(data_dir / f"collegeboard{str(yr)[-2:]}pub.csv", nrows=0)
    cols = (
        hdr.columns
           .str.strip()
           .str.lower()
           .str.replace(" ", "_")
           .map(lambda c: rename_map.get(c, c))
    )
    for c in cols:
        if c not in seen:
            seen.append(c)
all_cols = seen

# 3) Read & normalize each year's data
dfs = []
for yr in years:
    df = pd.read_csv(data_dir / f"collegeboard{str(yr)[-2:]}pub.csv", low_memory=False)
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(" ", "_")
    )
    df = df.rename(columns=rename_map)
    df = df.loc[:, ~df.columns.duplicated()]
    df['year'] = yr
    df = df.reindex(columns=all_cols)
    dfs.append(df)

# 4) Concatenate & drop duplicate-named columns
cb = pd.concat(dfs, ignore_index=True)
cb = cb.loc[:, ~cb.columns.duplicated()]

# 5) Round floats to nullable Ints
float_cols = cb.select_dtypes(include='float').columns
cb[float_cols] = cb[float_cols].round().astype('Int64')

# 6) Parse date columns
date_cols = [c for c in cb.columns if c in ['birthdt','grad_date'] or c.endswith('_dt')]
for c in date_cols:
    cb[c] = pd.to_datetime(cb[c], errors='coerce')

# 7) Cast categorical variables
for c in ['sex','ethnic','blang','lea','instname']:
    if c in cb.columns:
        cb[c] = cb[c].astype('category')

# 8) Drop stray essay subscore columns
essay_cols = ['sat_er_hc','sat_er_mr','sat_ea_hc','sat_ea_mr','sat_ew_hc','sat_ew_mr']
cb = cb.drop(columns=[c for c in essay_cols if c in cb.columns])

# 9) Build school_id (without dropping rows yet)
cb['birthdt'] = cb['birthdt'].replace(r'^\s*$', np.nan, regex=True)
for col in ['lea','schlcode']:
    if col in cb.columns:
        cb[col] = cb[col].replace(['<NA>','nan'], np.nan)
cb['lea']      = cb['lea'].astype(str).str.strip()
cb['schlcode'] = cb['schlcode'].astype(str).str.zfill(3)
cb['school_id'] = cb['lea'] + '-' + cb['schlcode']

# 10) Drop rows where school_id is invalid/missing
drop_mask = cb['school_id'].str.contains(r'^(nan|NA)-|-(nan|NA)$', na=False)
cb = cb[~drop_mask]

# 11) Compute total MR SAT score
cb['sat_total_score_mr'] = cb['sat_ebrw_score_mr'] + cb['sat_math_score_mr']

# ── 1) Compute psat_best and its z-score (if you haven't already) ──
cb['psat_best'] = (
    cb['psat_nmsqt_total']
      .fillna(cb['psat_10_total'])
      .fillna(cb['psat_8_9_total'])
)
cuts = cb['psat_best'].dropna().quantile([0, .25, .5, .75, 1.0]).values

# 3) Create a 5-level categorical: Q1–Q4 or Missing
cb['psat_bin'] = pd.cut(
    cb['psat_best'],
    bins=cuts,
    labels=['Q1','Q2','Q3','Q4'],
    include_lowest=True
)

cb['psat_bin'] = cb['psat_bin'].cat.add_categories('Missing')
cb['psat_bin'] = cb['psat_bin'].fillna('Missing')

# 4) One-hot encode psat_bin (plus any other dummies)
psat_dummies = pd.get_dummies(cb['psat_bin'], prefix='psat')
cb = pd.concat([cb, psat_dummies], axis=1)

# Now you have psat_Q1, psat_Q2, …, psat_Missing
# You can drop psat_best and psat_best_z entirely if you like:
cb_demo = cb.drop(columns=['psat_best'])
# 1) Define the columns you want to keep
keep = [
    'school_id',
    'year',
    'mastid',
    'sat_total_score_mr',
    'sat_ebrw_score_mr',
    'sat_math_score_mr',
    'sat_asmt_dt_mr',
    'psat_bin'
]

# 2) Subset cb in-place and drop any rows with NA in those columns
cb = (
    cb[keep]
    .dropna()
    .reset_index(drop=True)
)

# 3) Verify
print("Columns now:", cb.columns.tolist())
print("Any missing values left?", cb.isna().any().any())
cb.head()

/var/folders/1v/d265l3ks51x5svspj6rx8y3c0000gn/T/ipykernel_92904/917083283.py:79: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  drop_mask = cb['school_id'].str.contains(r'^(nan|NA)-|-(nan|NA)$', na=False)


Columns now: ['school_id', 'year', 'mastid', 'sat_total_score_mr', 'sat_ebrw_score_mr', 'sat_math_score_mr', 'sat_asmt_dt_mr', 'psat_bin']
Any missing values left? False


,school_id,year,mastid,sat_total_score_mr,sat_ebrw_score_mr,sat_math_score_mr,sat_asmt_dt_mr,psat_bin
0,81A-000,2017,7800274,1040,540,500,2016-12-03,Q3
1,410-406,2017,8001562,1080,500,580,2017-06-03,Q3
2,81A-000,2017,7434460,1240,610,630,2016-12-03,Missing
3,920-436,2017,8128749,1150,590,560,2017-06-03,Missing
4,630-336,2017,7440340,1090,550,540,2017-05-06,Q3


In [3]:
cb.to_csv(data_dir/"collegeboard_clean.csv", index=False)

In [4]:
# 0) Setup
data_dir = Path("/Users/adamcartwright/ncerdc/Student Data/Testing Data")
years    = [2017, 2018, 2019, 2020, 2021, 2022, 2023]

# 1) Map raw names → cleaned names
rename_map = {
    'sat_erw_score_hc': 'sat_ebrw_score_hc',
    'sat_erw_score_mr': 'sat_ebrw_score_mr',
    'sat_ctsh_ss_hc':   'sat_math_score_hc',
    'sat_ctsh_ss_mr':   'sat_math_score_mr',
    'ital':             'italgr',
    'reporting_year':   'year'
}

# 2) Build master column list
seen = []
for yr in years:
    hdr = pd.read_csv(data_dir / f"collegeboard{str(yr)[-2:]}pub.csv", nrows=0)
    cols = (
        hdr.columns
           .str.strip()
           .str.lower()
           .str.replace(" ", "_")
           .map(lambda c: rename_map.get(c, c))
    )
    for c in cols:
        if c not in seen:
            seen.append(c)
all_cols = seen

# 3) Read & normalize each year's data
dfs = []
for yr in years:
    df = pd.read_csv(data_dir / f"collegeboard{str(yr)[-2:]}pub.csv", low_memory=False)
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(" ", "_")
    )
    df = df.rename(columns=rename_map)
    df = df.loc[:, ~df.columns.duplicated()]
    df['year'] = yr
    df = df.reindex(columns=all_cols)
    dfs.append(df)

# 4) Concatenate & drop duplicate-named columns
cb = pd.concat(dfs, ignore_index=True)
cb = cb.loc[:, ~cb.columns.duplicated()]

# ——— Now copy before mutating ———
cb_full = cb.copy()

# 5) Round floats to nullable Ints
float_cols = cb_full.select_dtypes(include='float').columns
cb_full[float_cols] = cb_full[float_cols].round().astype('Int64')

# 6) Parse date columns
date_cols = [c for c in cb_full.columns if c in ['birthdt','grad_date'] or c.endswith('_dt')]
for c in date_cols:
    cb_full[c] = pd.to_datetime(cb_full[c], errors='coerce')

# 7) Cast categorical variables
for c in ['sex','ethnic','blang','lea','instname']:
    if c in cb_full.columns:
        cb_full[c] = cb_full[c].astype('category')

# 8) Drop stray essay subscore columns
essay_cols = ['sat_er_hc','sat_er_mr','sat_ea_hc','sat_ea_mr','sat_ew_hc','sat_ew_mr']
cb_full = cb_full.drop(columns=[c for c in essay_cols if c in cb_full.columns])

# 9) Build school_id (without dropping yet)
cb_full['birthdt'] = cb_full['birthdt'].replace(r'^\s*$', np.nan, regex=True)
for col in ['lea','schlcode']:
    if col in cb_full.columns:
        cb_full[col] = cb_full[col].replace(['<NA>','nan'], np.nan)
cb_full['lea']      = cb_full['lea'].astype(str).str.strip()
cb_full['schlcode'] = cb_full['schlcode'].astype(str).str.zfill(3)
cb_full['school_id'] = cb_full['lea'] + '-' + cb_full['schlcode']

# 10) Drop rows where school_id is invalid/missing
drop_mask = cb_full['school_id'].str.contains(r'^(nan|NA)-|-(nan|NA)$', na=False)
cb_full = cb_full[~drop_mask]

# 11) Compute total MR SAT score
cb_full['sat_total_score_mr'] = cb_full['sat_ebrw_score_mr'] + cb_full['sat_math_score_mr']

# —─ PSAT processing ──
cb_full['psat_best'] = (
    cb_full['psat_nmsqt_total']
      .fillna(cb_full['psat_10_total'])
      .fillna(cb_full['psat_8_9_total'])
)
cuts = cb_full['psat_best'].dropna().quantile([0, .25, .5, .75, 1.0]).values

cb_full['psat_bin'] = pd.cut(
    cb_full['psat_best'],
    bins=cuts,
    labels=['Q1','Q2','Q3','Q4'],
    include_lowest=True
).cat.add_categories('Missing').fillna('Missing')

# 12) (Optional) one-hot encode if you need later
psat_dummies = pd.get_dummies(cb_full['psat_bin'], prefix='psat')
cb_full = pd.concat([cb_full, psat_dummies], axis=1)

# 13) Drop the intermediate psat_best
cb_full = cb_full.drop(columns=['psat_best'])

keep = [
    'school_id',
    'year',
    'mastid',
    'psat_bin',            # categorical label
    'psat_Q1', 'psat_Q2', 'psat_Q3', 'psat_Q4', 'psat_Missing',  # one-hot dummies
    'sat_total_score_mr',
    'sat_ebrw_score_mr',
    'sat_math_score_mr',
    'sat_asmt_dt_mr'
]

cb_full = (
    cb_full
      .loc[:, keep]
      .dropna(subset=['school_id', 'year', 'mastid'])
      .reset_index(drop=True)
)

# 15) Verification
print("Original cb shape:",    cb.shape)
print("Processed cb_full shape:", cb_full.shape)
print("SAT‐takers in cb_full:", cb_full['sat_total_score_mr'].notna().sum())
print("Non‐takers in cb_full:", cb_full['sat_total_score_mr'].isna().sum())


/var/folders/1v/d265l3ks51x5svspj6rx8y3c0000gn/T/ipykernel_92904/3663650017.py:82: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  drop_mask = cb_full['school_id'].str.contains(r'^(nan|NA)-|-(nan|NA)$', na=False)


Original cb shape: (927820, 139)
Processed cb_full shape: (896030, 13)
SAT‐takers in cb_full: 309388
Non‐takers in cb_full: 586642


In [5]:
cb_full.to_csv(data_dir/"collegeboard_full.csv", index=False)